In [ ]:
!pip install torch transformers
import torch
from transformers import pipeline
import pandas as pd


In [ ]:
# load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/data.csv')

In [ ]:
df = data
# replace NaN values
df['About Product'] = df['About Product'].fillna('')

# initialize NLP pipeline
nlp_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad', tokenizer='bert-large-uncased')

# define a function to interact with the customer
def chatbot():
    print("Welcome to our store! How can I assist you today?")
    while True:
        user_input = input("Customer: ").strip()  # Get user input and strip()

        # exit condition
        if user_input.lower() == 'exit':
            print("Chatbot: Thank you for visiting us. Have a great day!")
            break

        # use the NLP pipeline to understand user input
        # join all 'About Product' descriptions as context
        context = ' '.join(df['About Product'].astype(str).tolist())

        answer = nlp_pipeline({
            'question': user_input,
            'context': context
        })

        # retrieve relevant products based on the answer
        if answer['score'] > 0.5:  # adjust the score threshold as needed
            product_name = answer['answer']
            relevant_products = df[df['Product Name'].str.lower().str.contains(product_name.lower())]

            if not relevant_products.empty:
                print(f"Chatbot: Here are some products related to '{product_name}':")
                for index, product in relevant_products.head(5).iterrows():
                    print(f"- Product Name: {product['Product Name']}")
                    print(f"  Category: {product['Category']}")
                    print(f"  Selling Price: ${product['Selling Price']}")
                    print()
            else:
                print(f"Chatbot: I'm sorry, we couldn't find products related to '{product_name}'.")
        else:
            print("Chatbot: I'm sorry, I couldn't understand your query. Could you please rephrase?")

        print("Chatbot: Is there anything else I can assist you with? If not, just say 'exit'.")

# let the chatbot begin
chatbot()